<a href="https://colab.research.google.com/github/hikmatfarhat-ndu/pytorch/blob/main/dl_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch 
import torchvision as vision

In [2]:
cifar10_train=vision.datasets.CIFAR10(".",download=True)
cifar10_test=vision.datasets.CIFAR10(".",download=True,train=False)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar-10-python.tar.gz to .
Files already downloaded and verified


In [3]:
img_train=torch.tensor(cifar10_train.data,dtype=torch.float32)/255.
img_test=torch.tensor(cifar10_test.data,dtype=torch.float32)/255.
# BCELoss requires the labels to be floats. Why??
label_train=torch.tensor(cifar10_train.targets,dtype=torch.float32)
label_test=torch.tensor(cifar10_test.targets,dtype=torch.float32)

flatten

In [4]:
img_train=img_train.reshape(50000,32*32*3)
img_test=img_test.reshape(10000,32*32*3)

In [5]:
#airplane=0,car=1,bird=2,cat=3,deer=4,dog=5,frog=6,horse=7,ship=8,truck=9
features=torch.tensor([0,1,8,9])
for i in range(label_train.shape[0]):
    if torch.isin(label_train[i],features):
        label_train[i]=1
    else:
        label_train[i]=0

for i in range(label_test.shape[0]):
    if torch.isin(label_test[i],features):
        label_test[i]=1
    else:
        label_test[i]=0        

In [6]:
torch.count_nonzero(label_train)

tensor(20000)

In [7]:
#samples=img_train.size()[1]
#weights=torch.rand(img_train.size()[1],requires_grad=True,dtype=torch.float32)
#weights.data/=samples
#bias=torch.tensor(0.,requires_grad=True,dtype=torch.float32)

In [8]:

import torch.nn as nn

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
  #bias is True by default. It is included here for illustration
    self.input_size=img_train.size()[1]
    self.output_size=1
    self.layer=nn.Linear(self.input_size,self.output_size,bias=True)
  def forward(self,x):  
    y_hat=self.layer(x)
    y_hat=torch.sigmoid(y_hat)
#   y_hat=torch.matmul(x,w)+b
#   y_hat=torch.sigmoid(y_hat)
    return y_hat

In [9]:
img_train=img_train.cuda()
img_test=img_test.cuda()
label_train=label_train.cuda()
label_test=label_test.cuda()

In [10]:
rate=0.015
model=Net().cuda()
import torch.optim as optim
optimizer=optim.SGD(model.parameters(),lr=0.015)
loss_fn=nn.BCELoss()

for i in range(1000):
  optimizer.zero_grad()
  y_hat=model(img_train)
  loss=loss_fn(y_hat.squeeze(),label_train)
  loss.backward()
  optimizer.step()
  if(i%100==0):
    print(loss)
  #weights.data-=rate*weights.grad.data
  #bias.data-=rate*bias.grad.data
  #bias.grad.data.zero_()
  #weights.grad.data.zero_()

tensor(0.7112, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.4725, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.4588, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.4524, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.4481, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.4449, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.4423, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.4402, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.4384, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.4368, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)


In [15]:
def predict(X):
    
    m = X.shape[0]
    print(m)
    Y_prediction = torch.zeros(m,1).cuda()
    
    # Compute vector "Y_hat" predicting
    #    the probabilities of a machine being present in the picture
    p_hat= model(X)   # compute activation

    print(p_hat.size())
    for i in range(p_hat.shape[0]):
        
        # Convert probabilities Y_hat[0,i] to actual predictions p[0,i]
        if p_hat[i]>=0.5:
            Y_prediction[i]=1
        else:
            Y_prediction[i]=0
    

    return Y_prediction

In [16]:
img_test.size()

torch.Size([10000, 3072])

In [17]:
zaza=predict(img_test)

10000
torch.Size([10000, 1])


In [18]:
Y_prediction_test = predict(img_test).squeeze()
Y_prediction_train = predict(img_train).squeeze()
print("train accuracy:"+str((100 - torch.mean(torch.abs(Y_prediction_train - label_train)) * 100)))
print("test accuracy:"+str((100 - torch.mean(torch.abs(Y_prediction_test - label_test)) * 100)))

10000
torch.Size([10000, 1])
50000
torch.Size([50000, 1])
train accuracy:tensor(81.3620, device='cuda:0')
test accuracy:tensor(81.1800, device='cuda:0')
